In [1]:
## setup
## TODO: move all under bird2022wheels

#!pip install ../input/birds-inference-pip-wheels/torchaudio-0.8.1-cp37-cp37m-manylinux1_x86_64.whl ../input/birds-inference-pip-wheels/torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl
#!pip install ../input/birds-inference-pip-wheels/audiomentations-0.16.0-py3-none-any.whl --no-index --no-deps
#!pip install ../input/birds-inference-pip-wheels/torchlibrosa-0.0.9-py3-none-any.whl --no-index --no-deps
!pip install ../input/nnaudio031/nnAudio-0.3.1-py3-none-any.whl
#!cp -r ../input/timmlatest ../working/timmlatest
#!pip install -U ../working/timmlatest
#!rm -rf ../working/timmlatest

# !pip install nnAudio

Processing /kaggle/input/nnaudio031/nnAudio-0.3.1-py3-none-any.whl


In [2]:
# !ls /kaggle/input/blef0503/
# !ls /kaggle/input/ef-v0/
# !ls /kaggle/input/birds-inference-pip-wheels/*
# !pip freeze|grep torch

In [3]:
## https://github.com/Selimonder/birdclef2022/
import os, sys, glob, math

os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

github_folder = "/kaggle/input/blef0508v3"
sys.path.append(github_folder)

In [4]:
import cv2
import gc
import torch
import librosa
import argparse, warnings

import numpy as np
import pandas as pd
import IPython.display as ipd

import zoo
from training.config import load_config
from training.datasets import BirdDatasetOOF


cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(0)

from tqdm import tqdm
from torch.utils.data import DataLoader

warnings.simplefilter("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


## models

In [5]:
def load_model(conf_path: str, weights_path: str, prefix: str, suffix: str, fold: int, to_device: bool = True):
    conf = load_config(conf_path)
    conf['encoder_params']['pretrained'] = False
    print(conf['encoder_params']['pretrained'], type(conf['encoder_params']['pretrained']))
    
    snapshot_name = "{}{}_{}_{}_{}".format(prefix, conf["network"], conf["encoder_params"]["encoder"], fold, suffix)
    weights_path = os.path.join(weights_path, snapshot_name)
    print(weights_path)
    
    model = zoo.__dict__[conf["network"]](**conf["encoder_params"])
    model = torch.nn.DataParallel(model)
    print("=> loading checkpoint '{}''".format(weights_path))
    checkpoint = torch.load(weights_path, map_location="cpu")
    print("epoch", checkpoint["epoch"])
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    if to_device: model.cuda()
    return model

models = []

## best only for now
suffixes = ["lb"]#, "f1_score" "last"]
folds    = 5

for i in [0]:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
                               weights_path = f"/kaggle/input/ef-v0-pretrain",
                               prefix       = "ef_v0",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

False <class 'bool'>
/kaggle/input/ef-v0-pretrain/ef_v0TimmClassifier_v3_tf_efficientnet_b0_0_lb
initing CLS features model 15 duration...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint '/kaggle/input/ef-v0-pretrain/ef_v0TimmClassifier_v3_tf_efficientnet_b0_0_lb''
epoch 19


In [6]:
len(models)
#models = [models[0]]

1

In [7]:
import torch

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(models[0])//1024

3593

## infer

In [8]:
CLASSES = "abethr1 abhori1 abythr1 afbfly1 afdfly1 afecuc1 affeag1 afgfly1 afghor1 afmdov1 afpfly1 afpkin1 afpwag1 afrgos1 afrgrp1 afrjac1 afrthr1 amesun2 augbuz1 bagwea1 barswa bawhor2 bawman1 bcbeat1 beasun2 bkctch1 bkfruw1 blacra1 blacuc1 blakit1 blaplo1 blbpuf2 blcapa2 blfbus1 blhgon1 blhher1 blksaw1 blnmou1 blnwea1 bltapa1 bltbar1 bltori1 blwlap1 brcale1 brcsta1 brctch1 brcwea1 brican1 brobab1 broman1 brosun1 brrwhe3 brtcha1 brubru1 brwwar1 bswdov1 btweye2 bubwar2 butapa1 cabgre1 carcha1 carwoo1 categr ccbeat1 chespa1 chewea1 chibat1 chtapa3 chucis1 cibwar1 cohmar1 colsun2 combul2 combuz1 comsan crefra2 crheag1 crohor1 darbar1 darter3 didcuc1 dotbar1 dutdov1 easmog1 eaywag1 edcsun3 egygoo equaka1 eswdov1 eubeat1 fatrav1 fatwid1 fislov1 fotdro5 gabgos2 gargan gbesta1 gnbcam2 gnhsun1 gobbun1 gobsta5 gobwea1 golher1 grbcam1 grccra1 grecor greegr grewoo2 grwpyt1 gryapa1 grywrw1 gybfis1 gycwar3 gyhbus1 gyhkin1 gyhneg1 gyhspa1 gytbar1 hadibi1 hamerk1 hartur1 helgui hipbab1 hoopoe huncis1 hunsun2 joygre1 kerspa2 klacuc1 kvbsun1 laudov1 lawgol lesmaw1 lessts1 libeat1 litegr litswi1 litwea1 loceag1 lotcor1 lotlap1 luebus1 mabeat1 macshr1 malkin1 marsto1 marsun2 mcptit1 meypar1 moccha1 mouwag1 ndcsun2 nobfly1 norbro1 norcro1 norfis1 norpuf1 nubwoo1 pabspa1 palfly2 palpri1 piecro1 piekin1 pitwhy purgre2 pygbat1 quailf1 ratcis1 raybar1 rbsrob1 rebfir2 rebhor1 reboxp1 reccor reccuc1 reedov1 refbar2 refcro1 reftin1 refwar2 rehblu1 rehwea1 reisee2 rerswa1 rewsta1 rindov rocmar2 rostur1 ruegls1 rufcha2 sacibi2 sccsun2 scrcha1 scthon1 shesta1 sichor1 sincis1 slbgre1 slcbou1 sltnig1 sobfly1 somgre1 somtit4 soucit1 soufis1 spemou2 spepig1 spewea1 spfbar1 spfwea1 spmthr1 spwlap1 squher1 strher strsee1 stusta1 subbus1 supsta1 tacsun1 tafpri1 tamdov1 thrnig1 trobou1 varsun2 vibsta2 vilwea1 vimwea1 walsta1 wbgbir1 wbrcha2 wbswea1 wfbeat1 whbcan1 whbcou1 whbcro2 whbtit5 whbwea1 whbwhe3 whcpri2 whctur2 wheslf1 whhsaw1 whihel1 whrshr1 witswa1 wlwwar wookin1 woosan wtbeat1 yebapa1 yebbar1 yebduc1 yebere1 yebgre1 yebsto1 yeccan1 yefcan yelbis1 yenspu1 yertin1 yesbar1 yespet1 yetgre1 yewgre1".split()
N_CLASS = len(CLASSES)

In [9]:
from torch.utils.data import Dataset, DataLoader

class BirdDataset(Dataset):
    def __init__(self, frame_length, sample_rate):
        self.frame_length = frame_length
        self.sample_rate = sample_rate
        self.data   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg")) 
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        Prepare audio clip for inference
        """
        fpath = self.data[idx]
        sample_rate = self.sample_rate
        frame_length = self.frame_length
        
        infer_frame_length = frame_length
        batch = {"wav_tensors": [], "end_times": []}

        waveform, sample_rate = librosa.load(fpath, sr=sample_rate, mono=True)
        n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))

        for seg_idx in range(n_parts): 
            end_time = (seg_idx + 1) * frame_length
            seg_wav  = waveform[(end_time*sample_rate)-(sample_rate*frame_length):end_time*sample_rate]

            wav_tensor = torch.from_numpy(seg_wav)

            if len(wav_tensor) == frame_length * sample_rate:
                batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
            elif len(wav_tensor) < frame_length * sample_rate:
                wav_tensor = torch.nn.functional.pad(wav_tensor, (0, (frame_length * sample_rate) - len(wav_tensor)))
                batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
            elif len(wav_tensor) > frame_length * sample_rate:
                wav_tensor = wav_tensor[:(frame_length * sample_rate)]
                batch["wav_tensors"].append(wav_tensor.unsqueeze(0))

            batch["end_times"].append(end_time)
        batch["wav_tensors"] = torch.stack(batch["wav_tensors"])
        return batch, n_parts, len(waveform)

@torch.no_grad()
def predict_clip(models, batch, n_parts, frame_length):
    preds = np.zeros([len(models), n_parts, N_CLASS])
    for m_idx, model in enumerate(models):
        with torch.cuda.amp.autocast():
            preds[m_idx] = model(batch["wav_tensors"], is_test=True)["logit"].sigmoid().cpu().numpy()
    return preds.mean(0) ## max by model    




In [10]:
frame_length = 5
infer_frame_length = 5
sample_rate  = 32000
test_ds = BirdDataset(frame_length, sample_rate)

# loader = DataLoader(
#     test_ds,
#     batch_size=16, 
#     num_workers=4,
#     drop_last=False,
#     shuffle=False,
#     pin_memory=True
# )

len(models), len(test_ds)

(1, 1)

In [11]:
%%time


@torch.no_grad()
def generate_preds():
    debug = False
    preds        = []
    # test_files   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg")) 
    scored_birds = np.array(CLASSES)
    # print("test file num:", len(test_files))
    
    for batch, n_parts, len_waveform in tqdm(test_ds):
        # file_id = os.path.basename(fpath).replace(".ogg", "")
        # batch, n_parts, len_waveform = prepare_clip(fpath, frame_length, sample_rate)
        clip_preds = predict_clip(models, batch, n_parts, frame_length)
        preds.append(clip_preds)
        
        ## switch to infer frame length
        n_parts_sub = math.ceil(len_waveform / int(infer_frame_length * sample_rate))
        clip_preds = np.array_split(clip_preds, n_parts_sub, axis=0)
        
                
    prob_array = np.array(preds)
    
    scored_bird_name2idx = {}
    for i, x in enumerate(scored_birds):
        scored_bird_name2idx[x] = i

    return prob_array, scored_bird_name2idx

prob_array, scored_bird_name2idx = generate_preds()
# del models
# gc.collect()
# torch.cuda.empty_cache()

100%|██████████| 1/1 [00:31<00:00, 31.86s/it]

CPU times: user 19.5 s, sys: 11.2 s, total: 30.6 s
Wall time: 31.9 s


In [12]:
prob_array.shape

(1, 120, 264)

----

In [13]:
def generate_file_ids():
    test_files   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg"))
    
    file_ids = []
    
    for fpath in test_files:
        file_id = os.path.basename(fpath).replace(".ogg", "")
        file_ids.append(file_id)
        
    return file_ids

file_ids = generate_file_ids()

In [14]:
## fix aniani
avg_preds = prob_array

avg_preds.shape

(1, 120, 264)

In [15]:
submission   = []
infer_frame_length = 5

for clip_preds, file_id in zip(avg_preds, file_ids):
    for frame_idx, pred in enumerate(clip_preds):
        end_time = (frame_idx + 1) * infer_frame_length
        d = {
            "row_id": f"{file_id}_{end_time}",
            # "target": pred[bi] > threshold,
        }
        for bi, bird in enumerate(CLASSES):
            d[bird] = pred[bi]
        submission.append(d)

In [16]:
%cd /kaggle/working

/kaggle/working


In [17]:
df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")

In [18]:
print(f"""
total rows : {len(df_submission)}
""")

df_submission.head()


total rows : 120



,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,afmdov1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
row_id,,,,,,,,,,,,,,,,,,,,,
soundscape_29201_5,0.012039,0.023610,0.040450,0.001910,0.007114,0.008289,0.008643,0.001818,0.010638,0.022225,...,0.000781,0.000816,0.006207,0.002168,0.016216,0.007601,0.004386,0.002177,0.006396,0.005333
soundscape_29201_10,0.036085,0.154867,0.016139,0.014758,0.008715,0.010163,0.035383,0.002671,0.009526,0.066254,...,0.001218,0.004391,0.022722,0.030288,0.010297,0.015590,0.007266,0.004634,0.003712,0.009591
soundscape_29201_15,0.093376,0.097572,0.060324,0.020133,0.009716,0.023378,0.090933,0.032717,0.012446,0.048878,...,0.005044,0.007839,0.023232,0.035310,0.028905,0.027866,0.024328,0.022154,0.034690,0.046711
soundscape_29201_20,0.002254,0.006189,0.007883,0.004370,0.010233,0.004951,0.005774,0.002555,0.008487,0.001456,...,0.000490,0.009187,0.006193,0.007686,0.004523,0.002345,0.006601,0.005127,0.004941,0.006406
soundscape_29201_25,0.004221,0.019556,0.023274,0.003951,0.025711,0.009332,0.006246,0.005239,0.008964,0.003156,...,0.002337,0.004589,0.011228,0.026581,0.005637,0.010578,0.008148,0.011642,0.005768,0.006329
